<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [13]:
!pip3 install beautifulsoup4
!pip3 install requests

In [14]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [15]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [16]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [17]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# URL of the Falcon 9 Launch Wiki page (as provided)
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

# Perform the HTTP GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    print("Successfully fetched the Falcon 9 Launch Wiki page.")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")


Successfully fetched the Falcon 9 Launch Wiki page.


Create a `BeautifulSoup` object from the HTML `response`


In [18]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
from bs4 import BeautifulSoup
# Assuming you have already fetched the HTML response and stored it in the 'response' object
html_content = response.text

# Create a BeautifulSoup object
soup = BeautifulSoup(html_content, 'html.parser')

# Print the type to verify if the BeautifulSoup object was created properly
print(type(soup))


<class 'bs4.BeautifulSoup'>


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [19]:
# Use soup.title attribute
# Assuming you have already created a BeautifulSoup object named 'soup'

# Print the page title to verify the BeautifulSoup object was created properly
page_title = soup.title.string
print("Page Title:", page_title)


Page Title: List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [20]:
# Find all table elements in the HTML
html_tables = soup.find_all('table')

# Print the number of tables found
print(f"Number of tables found on the page: {len(html_tables)}")

# Assume the third table is the one we are interested in (as observed)
if len(html_tables) >= 3:
    first_launch_table = html_tables[2]
else:
    print("The page does not contain three tables.")

# Extract column names from the table header
column_names = []

for th in first_launch_table.find_all('th'):
    column_name = th.get_text(strip=True)
    if column_name:  # Append the column name if it's not empty
        column_names.append(column_name)

# Print the extracted column names
print("Extracted Column Names:", column_names)


Number of tables found on the page: 25
Extracted Column Names: ['Flight No.', 'Date andtime (UTC)', 'Version,Booster[b]', 'Launch site', 'Payload[c]', 'Payload mass', 'Orbit', 'Customer', 'Launchoutcome', 'Boosterlanding', '1', '2', '3', '4', '5', '6', '7']


Starting from the third table is our target table contains the actual launch records.


In [22]:
# Ensure that the list has at least three tables
if len(html_tables) >= 3:
    # Access the third table (index 2, because list indices start at 0)
    third_table = html_tables[2]

    # Print the content of the third table
    print(third_table.prettify())
else:
    print("The page does not contain three tables.")


<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
 <tbody>
  <tr>
   <th scope="col">
    Flight No.
   </th>
   <th scope="col">
    Date and
    <br/>
    time (
    <a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">
     UTC
    </a>
    )
   </th>
   <th scope="col">
    <a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">
     Version,
     <br/>
     Booster
    </a>
    <sup class="reference" id="cite_ref-booster_11-0">
     <a href="#cite_note-booster-11">
      <span class="cite-bracket">
       [
      </span>
      b
      <span class="cite-bracket">
       ]
      </span>
     </a>
    </sup>
   </th>
   <th scope="col">
    Launch site
   </th>
   <th scope="col">
    Payload
    <sup class="reference" id="cite_ref-Dragon_12-0">
     <a href="#cite_note-Dragon-12">
      <span class="cite-bracket">
       [
      </span>
      c
      <span class="cite-bracket">
       ]
  

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [23]:
def extract_column_from_header(row):
    """
    This function processes the header cell (th element) to extract the column name.
    It removes unnecessary elements like <br>, <a>, <sup> and joins the contents into a single string.
    """
    if row.br:
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    column_name = ' '.join(row.contents)
    
    # Filter out digits and empty names
    if not(column_name.strip().isdigit()):
        column_name = column_name.strip()
        return column_name

# Assuming `first_launch_table` is the table you are working with

# List to store extracted column names
column_names = []

# Iterate through each <th> element in the table header
for th in first_launch_table.find_all('th'):
    # Apply the extract_column_from_header function to get the column name
    column_name = extract_column_from_header(th)
    
    # Append the column name to the list if it's not empty
    if column_name:
        column_names.append(column_name)

# Print the extracted column names
print("Extracted Column Names:", column_names)


Extracted Column Names: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


Check the extracted column names


In [24]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [39]:
# Assuming refined_column_names is the list of column names extracted and refined in the previous task
refined_column_names = ['Flight No.', 'Date and Time (UTC)', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']

# Initialize an empty dictionary with keys from the refined column names
launch_dict = {key: [] for key in refined_column_names}

# Assuming first_launch_table is the table you are working with
for row in first_launch_table.find_all('tr'):
    cells = row.find_all('td')
    
    # Skip rows that don't have enough cells
    if len(cells) < len(refined_column_names):
        continue
    
    # Append the extracted data to the appropriate list in the dictionary
    launch_dict['Flight No.'].append(cells[0].get_text(strip=True))
    
    # Extract date and time, assuming they are stored together in the second cell
    date_time_values = cells[1].get_text(strip=True).split()  # Splitting in case date and time are together
    if len(date_time_values) >= 2:
        date = date_time_values[0]
        time = date_time_values[1]
    else:
        date = date_time_values[0]
        time = ''  # Handle cases where time might be missing
    
    launch_dict['Date and Time (UTC)'].append(f"{date} {time}")
    
    launch_dict['Launch site'].append(cells[2].get_text(strip=True))
    launch_dict['Payload'].append(cells[3].get_text(strip=True))
    
    # Get the payload mass, handling missing or unusual entries
    payload_mass = cells[4].get_text(strip=True).replace("kg", "").strip()
    launch_dict['Payload mass'].append(payload_mass if payload_mass else 'N/A')
    
    launch_dict['Orbit'].append(cells[5].get_text(strip=True))
    launch_dict['Customer'].append(cells[6].get_text(strip=True))
    launch_dict['Launch outcome'].append(cells[7].get_text(strip=True))

import pandas as pd

# Convert the dictionary into a DataFrame
df_launches = pd.DataFrame(launch_dict)

# Display the first few rows of the DataFrame to verify the extraction
print(df_launches.head())


                  Flight No.   Date and Time (UTC)   Launch site  \
0          4 June 2010,18:45  F9 v1.0[7]B0003.1[8]  CCAFS,SLC-40   
1  8 December 2010,15:43[13]  F9 v1.0[7]B0004.1[8]  CCAFS,SLC-40   
2      22 May 2012,07:44[17]  F9 v1.0[7]B0005.1[8]  CCAFS,SLC-40   
3   8 October 2012,00:35[21]  F9 v1.0[7]B0006.1[8]  CCAFS,SLC-40   
4         1 March 2013,15:10  F9 v1.0[7]B0007.1[8]  CCAFS,SLC-40   

                                  Payload         Payload mass     Orbit  \
0    Dragon Spacecraft Qualification Unit                  N/A       LEO   
1       Dragondemo flight C1(Dragon C101)                  N/A  LEO(ISS)   
2  Dragondemo flight C2+[18](Dragon C102)  525  (1,157 lb)[19]  LEO(ISS)   
3           SpaceX CRS-1[22](Dragon C103)   4,700  (10,400 lb)  LEO(ISS)   
4           SpaceX CRS-2[22](Dragon C104)   4,877  (10,752 lb)  LEO(ISS)   

        Customer Launch outcome  
0         SpaceX        Success  
1  NASA(COTS)NRO     Success[9]  
2     NASA(COTS)    Success[20] 

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [43]:
# Helper functions provided earlier
def extract_date_time(table_cell):
    """Extract and clean the date and time from a table cell."""
    return [data_time.strip() for data_time in list(table_cell.strings)][0:2]

def extract_booster_version(table_cell):
    """Extract and clean the booster version from a table cell."""
    out = ''.join([booster_version for i, booster_version in enumerate(table_cell.strings) if i % 2 == 0][0:-1])
    return out

def extract_landing_status(table_cell):
    """Extract and clean the landing status from a table cell."""
    out = [i for i in table_cell.strings][0]
    return out

def extract_mass(table_cell):
    """Extract and clean the payload mass from a table cell."""
    mass = unicodedata.normalize("NFKD", table_cell.text).strip()
    if "kg" in mass:
        return mass[:mass.find("kg") + 2].strip()
    else:
        return mass or "N/A"

# Initialize an empty dictionary for storing the launch data
launch_dict = {
    "Flight No.": [],
    "Date and Time (UTC)": [],
    "Version Booster": [],
    "Launch site": [],
    "Payload": [],
    "Payload mass": [],
    "Orbit": [],
    "Customer": [],
    "Launch outcome": [],
    "Booster landing": []
}

# Iterate over each row in the HTML table and extract data
for row in first_launch_table.find_all('tr'):
    cells = row.find_all('td')

    # Skip rows that don't have enough cells
    if len(cells) < 8:  # Adjust the minimum number of expected cells
        continue

    # Extract and append data to launch_dict with safety checks
    launch_dict["Flight No."].append(cells[0].get_text(strip=True) if len(cells) > 0 else 'N/A')

    # Date and Time
    date_time = extract_date_time(cells[1]) if len(cells) > 1 else ['N/A', 'N/A']
    launch_dict["Date and Time (UTC)"].append(' '.join(date_time))

    # Version Booster
    launch_dict["Version Booster"].append(extract_booster_version(cells[2]) if len(cells) > 2 else 'N/A')

    # Launch site
    launch_dict["Launch site"].append(cells[3].get_text(strip=True) if len(cells) > 3 else 'N/A')

    # Payload
    launch_dict["Payload"].append(cells[4].get_text(strip=True) if len(cells) > 4 else 'N/A')

    # Payload mass
    launch_dict["Payload mass"].append(extract_mass(cells[5]) if len(cells) > 5 else 'N/A')

    # Orbit
    launch_dict["Orbit"].append(cells[6].get_text(strip=True) if len(cells) > 6 else 'N/A')

    # Customer
    launch_dict["Customer"].append(cells[7].get_text(strip=True) if len(cells) > 7 else 'N/A')

    # Launch outcome
    launch_dict["Launch outcome"].append(cells[8].get_text(strip=True) if len(cells) > 8 else 'N/A')

    # Booster landing
    launch_dict["Booster landing"].append(extract_landing_status(cells[9]) if len(cells) > 9 else 'N/A')

# Convert the dictionary into a DataFrame
df_launches = pd.DataFrame(launch_dict)

# Display the first few rows of the DataFrame to verify the extraction
print(df_launches.head())


                  Flight No. Date and Time (UTC) Version Booster  \
0          4 June 2010,18:45           F9 v1.0 [           CCAFS   
1  8 December 2010,15:43[13]           F9 v1.0 [           CCAFS   
2      22 May 2012,07:44[17]           F9 v1.0 [           CCAFS   
3   8 October 2012,00:35[21]           F9 v1.0 [           CCAFS   
4         1 March 2013,15:10           F9 v1.0 [           CCAFS   

                              Launch site                Payload Payload mass  \
0    Dragon Spacecraft Qualification Unit                                 LEO   
1       Dragondemo flight C1(Dragon C101)                           LEO (ISS)   
2  Dragondemo flight C2+[18](Dragon C102)  525 kg (1,157 lb)[19]    LEO (ISS)   
3           SpaceX CRS-1[22](Dragon C103)   4,700 kg (10,400 lb)    LEO (ISS)   
4           SpaceX CRS-2[22](Dragon C104)   4,877 kg (10,752 lb)    LEO (ISS)   

           Orbit     Customer             Launch outcome Booster landing  
0         SpaceX      Success

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [49]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


In [52]:
# Specify the file name and path where you want to save the CSV
csv_file_name = "falcon9_launches.csv"

# Export the DataFrame to a CSV file
df_launches.to_csv(csv_file_name, index=False)

# Confirm the file has been saved
print(f"DataFrame exported successfully to {csv_file_name}")


DataFrame exported successfully to falcon9_launches.csv


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
